In [1]:
import pandas as pd
import numpy as np
from HFCs_bmimAc_PR import configuration
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest
from pyomo.opt import TerminationCondition, SolverStatus

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import Flash
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
m1 = ConcreteModel()

m1.fs = FlowsheetBlock(default={"dynamic": False})

m1.fs.properties = GenericParameterBlock(default=configuration)

m1.fs.F101 = Flash(default={"property_package": m1.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

In [3]:
data = pd.read_csv('R32_bmimAc_10_2.csv')
data2 = pd.read_csv('R125_bmimAc_10_2.csv')

In [4]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [5]:
# Make outlet arrays
IL_x = np.zeros(len(data["1"]))
IL_y = np.zeros(len(data["1"]))
R32_x = np.zeros(len(data["1"]))
R32_y = np.zeros(len(data["1"]))
R125_x = np.zeros(len(data["1"]))
R125_y = np.zeros(len(data["1"]))

alpha1 = np.zeros(len(data["1"]))

m1.fs.F101.inlet.flow_mol.fix(10)
m1.fs.F101.inlet.temperature.fix(298)
m1.fs.F101.inlet.mole_frac_comp[0, "bmimAc"].fix(0.5) 
m1.fs.F101.inlet.pressure.fix(100000)
m1.fs.F101.vap_outlet.pressure.fix(100000)
m1.fs.F101.vap_outlet.temperature.fix(298.15)

for i in range(len(data["0"])):
    
    m1.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.009)
    m1.fs.properties.PR_kappa_A['R125', 'R32'].fix(0.007)
    m1.fs.properties.PR_kappa_A['R32', 'bmimAc'].fix(data["1"].iloc[i])
    m1.fs.properties.PR_kappa_A['bmimAc', 'R32'].fix(data["0"].iloc[i])
    m1.fs.properties.PR_kappa_A['bmimAc', 'R125'].fix(data2["0"].iloc[i])
    m1.fs.properties.PR_kappa_A['R125', 'bmimAc'].fix(data2["1"].iloc[i])

    m1.fs.F101.inlet.flow_mol.fix(10)
    m1.fs.F101.inlet.temperature.fix(298)
    m1.fs.F101.inlet.mole_frac_comp[0, "bmimAc"].fix(0.7) 
    m1.fs.F101.inlet.mole_frac_comp[0, "R32"].fix(0.2) 
    m1.fs.F101.inlet.mole_frac_comp[0, "R125"].fix(0.1)

    m1.fs.F101.inlet.pressure.fix(100000)

    m1.fs.F101.vap_outlet.pressure.fix(100000)
    m1.fs.F101.vap_outlet.temperature.fix(298.15)

    try:
        m1.fs.F101.initialize(outlvl=idaeslog.CRITICAL)              

        # solve the model
        status = solver.solve(m1, tee = False)

        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):

            IL_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "bmimAc"])
            IL_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "bmimAc"])
            R32_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "R32"]) 
            R32_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "R32"])
            R125_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "R125"])
            R125_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "R125"])

            print(i)
            
            alpha1[i] = (R125_y[i]/R125_x[i])/(R32_y[i]/R32_x[i])

    except ValueError:
        alpha1[i] = 'NaN'
       
        
results = pd.DataFrame(alpha1)
results.to_csv('bmimAc_alpha_10_2.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [6]:
print(alpha1)

[0.50666968 0.6409963  0.5139347  0.64334081 0.60315734 0.71001854
 0.5191886  0.59918021 0.54220208 0.6429129  0.64634292 0.64418411
 0.65222515 0.64129427 0.64496486 0.51798859 0.5061038  0.63969415
 0.47460758 0.64656284 0.55141563 0.59228057 0.56165486 0.55899527
 0.66857836 0.53447146 0.52290014 0.60442118 0.6954789  0.51584169
 0.63803988 0.59769205 0.52753785 0.62198935 0.63680099 0.58671096
 0.54308569 0.64343516 0.64414106 0.64509478 0.59243785 0.54864881
 0.59860014 0.55574312 0.59872791 0.56687646 0.509882   0.55444553
 0.58602094 0.50244031 0.62961653 0.51612897 0.48234783 0.63940727
 0.63709566 0.56375295 0.58778521 0.52090744 0.58165279 0.73434262
 0.57747515 0.61936907 0.5991628  0.51788541 0.59648216 0.58697291
 0.61771434 0.61038302 0.56661561 0.64586641 0.66637833 0.61917658
 0.50997273 0.45948513 0.54030753 0.5934375  0.64403554 0.6247551
 0.56925822 0.6441844  0.64516    0.64484774 0.5873709  0.50440233
 0.6449184  0.6704262  0.59320642 0.55976025 0.53906756 0.64387